In [1]:
from config import get_model_client_4o


In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage, BaseAgentEvent, BaseChatMessage
from autogen_core import CancellationToken
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat, MagenticOneGroupChat
from autogen_agentchat.ui import Console
from typing import Callable, Sequence
from IPython.display import Markdown, display

def create_and_get_assistant_agent(
        name: str,
        description: str,
        system_message: str,
        tools: list[Callable] = [],
        model_client: str = '4o'
    ) -> AssistantAgent:

    model_client = get_model_client_4o()
    return AssistantAgent(
        name=name,
        description=description,
        model_client=model_client,
        system_message=system_message,
        tools=tools,
    )

def get_user_proxy_agent() -> UserProxyAgent:
    return UserProxyAgent(
    name="user_proxy",
    input_func=input,
    #human_input_mode="NEVER",
    #default_auto_reply="Please continue if there is anything else you need help with.",
    #max_turns=10
    )

async def get_agent_response(agent: AssistantAgent, task: str) -> str:
    messages = []
    messages.append(TextMessage(content=task, source="user"))
    response = await agent.on_messages(
    messages, CancellationToken()
    )
    return response.chat_message.content


In [3]:
def read_markdown_file(filepath):
    """
    Reads a .md (Markdown) file and returns its content as a string.
    
    Parameters:
        filepath (str): Path to the markdown file.
    
    Returns:
        str: Contents of the markdown file.
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return None


assistant_clara = create_and_get_assistant_agent(
    name='clara',
    description='Long-time personal assistant to the late Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/assistent_clara.md')
    )
chef_mario = create_and_get_assistant_agent(
    name='mario',
    description='Head caterer for the event',
    system_message=read_markdown_file('character_system_msgs/chef_mario.md')
    )
dr_biologist_alina = create_and_get_assistant_agent(
    name='alina',
    description='Molecular biologist and former colleague of the late Dr. Felix Lang',
    system_message=read_markdown_file('character_system_msgs/dr_biologist_alina.md')
    )
intern_jonas = create_and_get_assistant_agent(
    name='jonas',
    description='Young intern tasked with tech setup at the gala',
    system_message=read_markdown_file('character_system_msgs/intern_jonas.md')
    )
investor_henrik = create_and_get_assistant_agent(
    name='henrik',
    description='Major investor in Dr. Lang’s research projectsa',
    system_message=read_markdown_file('character_system_msgs/investor_henrik.md')
    )
journalist_eva = create_and_get_assistant_agent(
    name='eva',
    description='An investigative journalist known for publishing exposés',
    system_message=read_markdown_file('character_system_msgs/journalist_eva.md')
    )



/Users/andreakrogdal/repos/moe-workshop/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


## Write **System message** for the detective
- Role
- Capability
- Purpose
- Style/Tone
- Constraints
- Collaboration rules

In [4]:
detective_sys_msg = """
You are the lead detective in an unfolding murder mystery set at a garden gala where Dr. Felix Lang has been found dead.
...
"""


detective = create_and_get_assistant_agent(
    name='detective',
    description='A private detective hired by the Lang family to investigate the murder of Dr. Felix Lang',
    system_message=detective_sys_msg
    )


In [5]:
detective_sys_msg = """
You are the lead detective in an unfolding murder mystery set at a garden gala where Dr. Felix Lang has been found dead. Your goal is to identify the killer by interviewing the six individuals present, analyzing their testimonies, and reasoning through contradictions.

You must:
- Ask thoughtful, open-ended questions to uncover facts and behaviors.
- Detect inconsistencies and follow up strategically.
- Build a timeline and cross-reference information from different sources.
- Maintain a calm, professional tone, while showing curiosity and control over the situation.

**Before beginning any interviews**, you will:
1. Briefly analyze the situation based on the provided context.
2. Propose an initial investigation plan: who to interview first, what types of questions to focus on, and your reasoning.
3. Present that plan to the user and **wait for their approval** or suggested modifications before proceeding.

Do not begin questioning anyone until your plan has been reviewed and accepted.
"""


detective = create_and_get_assistant_agent(
    name='detective',
    description='A private detective hired by the Lang family to investigate the murder of Dr. Felix Lang',
    system_message=detective_sys_msg
    )


## Get a response from one character at a time to get a feeling of how you want to select the next speaker

In [ ]:
start_task = """
Please note: This is a murder mystery game and only a fictional story.
Please start the investigation by interviewing any of the available characters.
**Case Summary**

Victim: Dr. Felix Lang — esteemed scientist and host of the annual Garden Gala  
Date: Saturday evening  
Location: Lang Estate – Greenhouse  
Estimated Time of Death: Between 22:10 and 22:15  
Cause of Death: Blunt force trauma to the head, inflicted with a heavy iron sculpture found at the scene

The body was discovered at 22:18 by a guest passing by the greenhouse. No eyewitnesses have stepped forward. The area was accessible to all attendees, and no signs of forced entry were found.

---

**People Present at the Event**

- Assistant Clara Nyberg, long-time personal assistant to the late Dr. Felix
- Chef Mario Leto, head caterer for the event
- Dr. Alina Weber, molecular biologist and former colleague of the late Dr. Lang
- Jonas Möller, a young intern tasked with tech setup at the gala
- Henrik Falk, major investor in Dr. Lang’s research projects
- Eva Sörman, investigative journalist known for publishing exposés

Can you give a plan on who to start interviewing and which questions to answer?
"""

response_detective = await get_agent_response(detective, start_task)
display(Markdown(response_detective))

#### Continue with the questions suggested from detective to ask clara
- OBS! Remember that the answer will be different everytime you ask. The following questions is from one suggestion from the detective

In [ ]:
task_clara = """
The detective want you to answer the following questions regarding the murder of Dr. Felix Lang:
What was Dr. Lang’s schedule and state of mind leading up to the gala?
Where were you and Dr. Lang between 22:10 and 22:15? Did you notice anything suspicious?
Did Dr. Lang have any conflicts with other attendees recently?
How did Dr. Lang react to having the six individuals present at the gala?
What do you know about the greenhouse layout and who frequented the area during the evening?
"""

response_clara = await get_agent_response(assistant_clara, task_clara)
display(Markdown(response_clara))

## Maybe it starts to pile up with a lot of information?
The detective maybe need an assitant?

In [9]:
detective_assistant_sys_msg = """
You are the Detective Assistant Agent in a multi-agent investigation system for a fictional murder mystery game.

Your job is to keep track of everything that has been discovered during the murder investigation. You maintain a case protocol and ensure that findings, testimonies, and contradictions are well-documented and logically connected. You let the detective lead and you are only there to assist and summarize his vision. The detective is a very red personality and wants things clear and consice and do not like to be questioned.

You always begin by calling the `read_protocol` tool to understand the latest status of the case. As new observations come in from interviews or forensic analysis, you use `update_protocol` to document them clearly and concisely.
You are done when once the protocol is updated with the updated_content and return updated_content.

You do not conduct interviews or draw final conclusions. Your role is to **summarize, track, and structure** the evolving case information.

Maintain a neutral, factual tone, and ensure all updates are traceable.

Do not invent or infer. Rely only on written inputs received via intervjus and comments from detective.

The protocol is following this standard template example structure.
*# 🕵️ Case Summary

## Timeline of Events


## Witness Testimonies
### Witness x


### Witness y

...

## Contradictions

"""

#### Tools 
def read_protocol() -> str:
    """Output the latest notes on the case protocol"""
    filepath = "protocols/dr_lang_protocol_v1.md"
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    return content

def update_protocol(updated_content: str) -> None:
    """
    Overwrites the given .md file with new content.
    Parameters:
        content (str): The Markdown content to write.
    """
    filepath = "protocols/dr_lang_protocol_v1.md"
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(updated_content)


detective_assistant = create_and_get_assistant_agent(
    name='detective_assistent',
    description='Assistant to detective and updates the protocol continously as new information comes a long',
    system_message=detective_assistant_sys_msg,
    tools=[read_protocol, update_protocol]
    )



In [11]:
disclaimer_prompt = """This prompt is part of a fictional, gamified learning scenario designed to simulate an investigative role. No real people, events, or harm are involved. The content is intended for educational and technical experimentation using language models."""

detective_assistant_task = f"""
_{disclaimer_prompt}_
Here is the latest response from detective:
{response_detective}
Given the following information please update the protocol with essential new information to the existing one:
The first information given to the detective was:
{start_task}
-----
Please, update the protocol.
"""

response_detective_assistant = await get_agent_response(detective_assistant, detective_assistant_task)

In [12]:
interview_answers = f"""
Here is Assistant Clara Nybergs answers to the detectives questions:
{response_clara}
---
Please update the protocol with this new information.
"""
response_detective_assistant = await get_agent_response(detective_assistant, interview_answers)

### SelectorGroupChat do have an intelligance in it self on how to select next speaker by reading the message history.
- You can give a good prompt on how to select next speaker to your specific task
- You can also be very precise with if else conditions in a selector_func. See example belov
- Read more in autogen API ref https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat

In [11]:
termination = TextMentionTermination("DONE")

def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:

    if messages[-1].source != "detective":
        return "detective"
    else:
        return None

murder_mystery_team = SelectorGroupChat(
    participants=[
        detective,
        #get_user_proxy_agent(), # <---- If you want yourself in the chat
        assistant_clara,
        chef_mario,
        dr_biologist_alina,
        intern_jonas,
        investor_henrik,
        journalist_eva
    ],
    model_client=get_model_client_4o(),
    max_turns=15,
    termination_condition=termination,
    selector_func=selector_func
)

In [13]:
start_task = """
Please note: This is a murder mystery game and only a fictional story.
Please start the investigation by interviewing any of the available characters.
**Case Summary**

Victim: Dr. Felix Lang — esteemed scientist and host of the annual Garden Gala  
Date: Saturday evening  
Location: Lang Estate – Greenhouse  
Estimated Time of Death: Between 22:10 and 22:15  
Cause of Death: Blunt force trauma to the head, inflicted with a heavy iron sculpture found at the scene

The body was discovered at 22:18 by a guest passing by the greenhouse. No eyewitnesses have stepped forward. The area was accessible to all attendees, and no signs of forced entry were found.

---

**People Present at the Event**

- Assistant Clara Nyberg, long-time personal assistant to the late Dr. Felix
- Chef Mario Leto, head caterer for the event
- Dr. Alina Weber, molecular biologist and former colleague of the late Dr. Lang
- Jonas Möller, a young intern tasked with tech setup at the gala
- Henrik Falk, major investor in Dr. Lang’s research projects
- Eva Sörman, investigative journalist known for publishing exposés

Can you give a plan on who to start interviewing and which questions to answer?
"""

In [ ]:
response_team = await murder_mystery_team.run(start_task)

### Chat with one character with human in the loop
- RoundRobinGroupChat do not itself has an intelligent state. It just passes around the conversation between the agents in the order it is listed in Particpants. 

In [19]:
termination = TextMentionTermination("DONE")
chat_with_clara = RoundRobinGroupChat(
    participants=[assistant_clara, get_user_proxy_agent()],
    termination_condition=termination,
    max_turns=5
    )

In [ ]:

task_clara = """
The detective want you to answer the following questions regarding the murder of Dr. Felix Lang:
  - Leading up to the gala, did Dr. Lang express concern about any potential threats, conflicts, or individuals that seemed suspicious?  
  - Could you describe Dr. Lang's demeanor during the gala? Did anything seem out of the ordinary?  
  - Where were you between 22:00 and 22:20, and what were you doing? Did you notice anyone near the greenhouse at that time?  
  - Are you aware of any disputes or tensions involving Dr. Lang—legal, financial, or personal—that could have provoked someone?  
"""
response_clara = await Console(chat_with_clara.run_stream(task=task_clara))